In [1]:
import numpy as np
import tensorflow as tf
import requests
from skimage.transform import resize
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from glob import glob
import gradio as gr
import cv2
from PIL import Image
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
def preprocess_npy(npy):
#     npy = resize(npy,(128,128))
    npy = np.load(npy,allow_pickle=True)
    npy = np.reshape(npy, (1,512,512,1))
    return npy

In [4]:
def dice_coef(y_true, y_pred, smooth=1):
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    dice = (2. * intersection + smooth) / (union + smooth)
    return dice

def dice_coef_loss(y_true, y_pred):
    loss = 1 - dice_coef(y_true, y_pred)
    return loss


In [5]:
def segmentation(img_path):
    path = r"path to segmented model"
    img = preprocess_npy(img_path)
    model = load_model(path, custom_objects={'dice_coef_loss':dice_coef_loss, 'dice_coef':dice_coef})
    prediction = model.predict(img)
    return prediction

In [6]:
def classification(pred):
    model = load_model(r"path to the classification model")
    pred = resize(pred, (pred.shape[0],128,128,pred.shape[3]))
    class_pred = model.predict(pred)
    return np.argmax(class_pred)

In [7]:
def output_results(path):
    seg_pred = segmentation(path)
    class_pred = classification(seg_pred)
    if class_pred+1 in [1,2]:
        pred = 'Benign'
    elif class_pred+1 in [4,5]:
        pred = 'Malignant'
    elif class_pred+1 == 3:
        pred = 'Ambiguous'
    return final_process(seg_pred), pred

In [8]:
def final_process(pred):
#     pred = pred.astype('uint8')
    print(np.squeeze(pred).shape)
#     plt.imshow(np.squeeze(pred))
    return np.squeeze(pred)

In [9]:
interface = gr.Interface(
    fn = output_results,
    inputs = gr.components.Text(),
    outputs = [gr.components.Image(shape=(512,512), label="Segmentation Output"), gr.components.Text(label="Classification")]
)

In [10]:
interface.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 395ms/step
(512, 512)
1/1 [==============================] - 0s 147ms/step
(512, 512)
1/1 [==============================] - 0s 182ms/step
(512, 512)
1/1 [==============================] - 0s 173ms/step
(512, 512)
1/1 [==============================] - 0s 189ms/step
(512, 512)
1/1 [==============================] - 0s 169ms/step
(512, 512)
Keyboard interruption in main thread... closing server.
